1. History & Versioning
2. Time Travel
3. Vaccum

In [0]:
%sql
USE f1_demo;

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge
-- to check the history of updation of the table

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 2;
-- checking on version 1

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-07-31T10:15:28.000+00:00';
-- checking on timestamp


In [0]:
#to load the data from the timestamp
df = spark.read.format("delta").option("timestampAsOf", '2025-07-31T10:15:28.000+00:00').load("dbfs:/FileStore/demo/drivers_merge")

In [0]:
%sql
VACUUM f1_demo.drivers_merge
-- by default retains data upto 7 days
-- after running this command, can see the data under 7 days.

In [0]:
%sql
VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

--to show that we are sure doing this

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge
--still can see the table and the data

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-07-31T10:15:28.000+00:00';
--unable to see the data after vacuuming based on timestamp

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge
-- can still see the history,but lost the ability to go back to the data at that moment after doig vacumm

In [0]:
%sql
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

In [0]:
%sql 
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 3;
--still showing with id 1

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSION AS OF 3 src
   ON (tgt.driverId = src.driverId)
WHEN NOT MATCHED THEN
   INSERT *
-- merging version 3 to this table
-- merged that deleted row

In [0]:
%sql DESC HISTORY f1_demo.drivers_merge

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

Transation Logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING DELTA

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 1;

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 2;

In [0]:
%sql
DELETE FROM  f1_demo.drivers_txn
WHERE driverId = 1;

everytime we perform some operation on the file that is stored in the delta lake, then a new file gets added in the log folder that is present inside that file folder itself.
alWAYS CREATES a new parquet file everytime we do a new operation.

In [0]:
for driver_id in range(3, 20):
  spark.sql(f"""INSERT INTO f1_demo.drivers_txn
                SELECT * FROM f1_demo.drivers_merge
                WHERE driverId = {driver_id}""")
  
  #as we perform this operation like from 3 to 20, then along with json files for then it also creates a checkpoint file and this checkpoint is present after 9th in the case means as if we wanted to read from file 11th then read the data from checkpoint and then 10th and then 11th.

Convert parquet to delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_convert_to_delta (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING PARQUET

In [0]:
%sql
INSERT INTO f1_demo.drivers_convert_to_delta
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
CONVERT TO DELTA f1_demo.drivers_convert_to_delta
-- this will create a new delta_log file in the format as soon as the file format converts

In [0]:
df = spark.table("f1_demo.drivers_convert_to_delta")